# Original - Minimizes Errors (Turns out not-needed)

In [3]:
import numpy as np

# First defining a small number for float comparisons
EPSILON = 1e-9

def solve_single_constraint_lp(c, A_row, b_val, problem_type="min", debug=False):
    """
    Solves a linear programming problem with a SINGLE equality constraint
    using a simplified Simplex Tableau method (simpler version of Wikipedia explanation)

    Form: max/min c*x subject to A_row*x = b_val, x >= 0.

    Inputs:
        c (np.ndarray or list): 1D array/list of objective function coefficients
        A_row (np.ndarray or list): 1D array/list of the single constraint coefficients.
        b_val (float): The single constraint right-hand side value (must be >= 0).
        problem_type (str): "max" or "min" (default: "min").
        debug (bool): true = prints iteration details, just a bunch of work to ensure that it runs right (for me)

    Outputs (hopefully):
        tuple: (solution, objective_value)
               solution (a numpy array): Optimal variable values.
               objective_value (float): Optimal objective function value.
               Note: should return (None, None) if problem is unbounded.
    """

    # --- Input Validation and Setup ---
    c = np.array(c, dtype=float)
    A_row = np.array(A_row, dtype=float)
    b_val = float(b_val)

    num_vars = len(c)

    # If minimizing, convert to maximization internally
    c_orig = c.copy() # Keep original for final objective calculation if min
    if problem_type.lower() == "min":
        c = -c

    # --- Find Initial Basic Variable ---
    # Find the first variable with a positive coefficient in the constraint
    initial_basic_idx = -1
    for i in range(num_vars):
        if A_row[i] > EPSILON:
            initial_basic_idx = i
            break

    if initial_basic_idx == -1:
        # This case implies all A_row coefficients are <= 0.
        # If b_val > 0, the problem is infeasible.
        # If b_val = 0, only x=0 is possible if all A_row coeffs are 0,
        # or multiple solutions if some A_row coeffs are negative.
        # For simplicity under the assumption b>=0 and at least one ai>0:
        raise ValueError("Cannot find an initial basic variable (no positive coefficient in A_row).")

    if debug:
        print(f"INFO: Choosing x{initial_basic_idx+1} as initial basic variable.")

    # --- Build and Normalize Initial Tableau ---
    # Tableau: 2 rows (obj, constraint), N+1 columns (vars + RHS)
    tableau = np.zeros((2, num_vars + 1))

    # Fill constraint row (row 1) and normalize it
    pivot_val_initial = A_row[initial_basic_idx]
    tableau[1, :num_vars] = A_row / pivot_val_initial
    tableau[1, -1] = b_val / pivot_val_initial
    # Ensure the basic variable column has exactly 1 after division

    
    #tableau[1, initial_basic_idx] = 1.0

    # Fill objective row (row 0) - initially [-c | 0]
    tableau[0, :num_vars] = -c
    tableau[0, -1] = 0.0

    # Adjust objective row to make coefficient for basic variable zero
    multiplier = tableau[0, initial_basic_idx] # / tableau[1, initial_basic_idx] which is 1
    tableau[0, :] -= multiplier * tableau[1, :]
    # Ensure the basic variable coefficient is exactly zero
    
    #tableau[0, initial_basic_idx] = 0.0

    # Track the current basic variable index
    basic_var_idx = initial_basic_idx

    # --- Simplex Iterations ---
    iteration = 0
    max_iterations = 50 # Nothing should get past this at this level of problem, just a debug thing

    while iteration < max_iterations:
        if debug:
            print(f"\n--- Iteration {iteration} ---")
            var_labels = [f"x{i+1}" for i in range(num_vars)] + ["RHS"]
            basis_label = f"x{basic_var_idx+1}"
            print(f"Tableau (Basic Var: {basis_label}):")
            # Simple printout
            print("      " + " | ".join(f"{h:>6}" for h in var_labels))
            print(" z' | " + " | ".join(f"{v:>6.2f}" for v in tableau[0]))
            print(f" {basis_label:<3}| " + " | ".join(f"{v:>6.2f}" for v in tableau[1]))

        # 1. Check for Optimality (Maximization problem)
        obj_coeffs = tableau[0, :num_vars]
        all_non_negative = True
        for i in range(num_vars):
            if i != basic_var_idx and obj_coeffs[i] < -EPSILON:
                all_non_negative = False
                break
        if all_non_negative:
            if debug: print("\nOptimal solution found.")
            break

        # 2. Find Pivot Column (Entering Variable)
        # Most negative coefficient among non-basic variables
        pivot_col = -1
        min_coeff = -EPSILON
        for j in range(num_vars):
            if j != basic_var_idx and obj_coeffs[j] < min_coeff:
                min_coeff = obj_coeffs[j]
                pivot_col = j

        if pivot_col == -1:
            # Just for debugging saftey, this...theoretically should never be reached?
            break

        if debug:
            print(f"Pivot column (entering var): x{pivot_col+1} (index {pivot_col}) with coeff {obj_coeffs[pivot_col]:.2f}")

        # 3. Minimum Ratio Test (Find Leaving Variable / Pivot Row)
        # With only one constraint, the leaving variable is always the current basic variable.
        # We just need to check if the problem is unbounded.
        pivot_row = 1 # Always the constraint row
        entry_in_pivot_col = tableau[pivot_row, pivot_col]

        if entry_in_pivot_col <= EPSILON:
            # If the entry is non-positive, we can increase the entering
            # variable indefinitely without violating the constraint (since b>=0).
            print("Error: Problem is unbounded.")
            return None, None

        pivot_element = entry_in_pivot_col
        if debug:
            print(f"Pivot element (Row {pivot_row}, Col {pivot_col}): {pivot_element:.2f}")
            print(f"Leaving variable: x{basic_var_idx+1} (index {basic_var_idx})")


        # 4. Pivoting
        # a) Normalize the pivot row (row 1)
        tableau[pivot_row, :] /= pivot_element
        tableau[pivot_row, pivot_col] = 1.0 # Just making sure this is exactly 1

        # b) Eliminate entry in objective row (row 0)
        multiplier = tableau[0, pivot_col]
        tableau[0, :] -= multiplier * tableau[pivot_row, :]
        tableau[0, pivot_col] = 0.0 # same but 0

        # 5. Update Basis
        basic_var_idx = pivot_col # The entering variable becomes basic

        iteration += 1

    # --- End of Loop ---
    if iteration >= max_iterations:
        return None, None

    # --- Get Solution ---
    solution = np.zeros(num_vars)
    # The basic variable gets the right side value from the constraint row
    solution[basic_var_idx] = tableau[1, -1]

    # Get rid of small floating point errors 
    solution[np.abs(solution) < EPSILON] = 0.0

    # Final objective value (for the maximization problem)
    max_obj_value = tableau[0, -1]

    # If original problem was minimization, return negative of max value
    final_obj_value = -max_obj_value if problem_type.lower() == "min" else max_obj_value

    return solution, final_obj_value




# Removes all error minimization (no change in results)

In [4]:
import numpy as np

def solve_single_constraint_lp(c, A_row, b_val, problem_type="min", debug=False):
    """
    Solves a linear programming problem with a SINGLE equality constraint
    using a simplified Simplex Tableau method.
    """

    # --- Input Validation and Setup ---
    c = np.array(c, dtype=float)
    A_row = np.array(A_row, dtype=float)
    b_val = float(b_val)

    num_vars = len(c)

    # If minimizing, convert to maximization internally
    c_orig = c.copy()
    if problem_type.lower() == "min":
        c = -c

    # --- Find Initial Basic Variable ---
    initial_basic_idx = -1
    for i in range(num_vars):
        if A_row[i] > 0:  # No EPSILON used here
            initial_basic_idx = i
            break

    if initial_basic_idx == -1:
        raise ValueError("Cannot find an initial basic variable (no positive coefficient in A_row).")

    if debug:
        print(f"INFO: Choosing x{initial_basic_idx+1} as initial basic variable.")

    # --- Build and Normalize Initial Tableau ---
    tableau = np.zeros((2, num_vars + 1))

    pivot_val_initial = A_row[initial_basic_idx]
    tableau[1, :num_vars] = A_row / pivot_val_initial
    tableau[1, -1] = b_val / pivot_val_initial

    # Ensure the basic variable column has exactly 1
    tableau[1, initial_basic_idx] = 1.0

    tableau[0, :num_vars] = -c
    tableau[0, -1] = 0.0

    # Adjust objective row to eliminate the basic variable column
    multiplier = tableau[0, initial_basic_idx]
    tableau[0, :] -= multiplier * tableau[1, :]
    # No manual setting tableau[0, initial_basic_idx] = 0.0

    basic_var_idx = initial_basic_idx

    # --- Simplex Iterations ---
    iteration = 0
    max_iterations = 50

    while iteration < max_iterations:
        if debug:
            print(f"\n--- Iteration {iteration} ---")
            var_labels = [f"x{i+1}" for i in range(num_vars)] + ["RHS"]
            basis_label = f"x{basic_var_idx+1}"
            print(f"Tableau (Basic Var: {basis_label}):")
            print("      " + " | ".join(f"{h:>6}" for h in var_labels))
            print(" z' | " + " | ".join(f"{v:>6.6f}" for v in tableau[0]))
            print(f" {basis_label:<3}| " + " | ".join(f"{v:>6.6f}" for v in tableau[1]))

        # 1. Check for Optimality
        obj_coeffs = tableau[0, :num_vars]
        all_non_negative = True
        for i in range(num_vars):
            if i != basic_var_idx and obj_coeffs[i] < 0:  # No EPSILON check
                all_non_negative = False
                break
        if all_non_negative:
            if debug:
                print("\nOptimal solution found.")
            break

        # 2. Find Pivot Column (Entering Variable)
        pivot_col = -1
        min_coeff = 0
        for j in range(num_vars):
            if j != basic_var_idx and obj_coeffs[j] < min_coeff:
                min_coeff = obj_coeffs[j]
                pivot_col = j

        if pivot_col == -1:
            break  # Should not happen normally

        if debug:
            print(f"Pivot column (entering var): x{pivot_col+1} (index {pivot_col}) with coeff {obj_coeffs[pivot_col]:.6f}")

        # 3. Minimum Ratio Test (Find Leaving Variable)
        pivot_row = 1
        entry_in_pivot_col = tableau[pivot_row, pivot_col]

        if entry_in_pivot_col <= 0:  # No EPSILON safety
            print("Error: Problem is unbounded.")
            return None, None

        pivot_element = entry_in_pivot_col
        if debug:
            print(f"Pivot element (Row {pivot_row}, Col {pivot_col}): {pivot_element:.6f}")
            print(f"Leaving variable: x{basic_var_idx+1} (index {basic_var_idx})")

        # 4. Pivoting
        tableau[pivot_row, :] /= pivot_element
        # No forced pivot_col = 1.0 again (already handled by division)

        multiplier = tableau[0, pivot_col]
        tableau[0, :] -= multiplier * tableau[pivot_row, :]
        # No forced tableau[0, pivot_col] = 0.0

        # 5. Update Basis
        basic_var_idx = pivot_col

        iteration += 1

    # --- End of Loop ---
    if iteration >= max_iterations:
        return None, None

    # --- Get Solution ---
    solution = np.zeros(num_vars)
    solution[basic_var_idx] = tableau[1, -1]

    # No cleaning of small values

    max_obj_value = tableau[0, -1]
    final_obj_value = -max_obj_value if problem_type.lower() == "min" else max_obj_value

    return solution, final_obj_value


In [5]:
if __name__ == "__main__":

    print("--- Example 1: Original Problem ---")
    c1 = [1.0, 2.0, 3.0]
    A1_row = [1.0, 1.0, 1.0]
    b1_val = 1.0
    solution1, objective_value1 = solve_single_constraint_lp(c1, A1_row, b1_val, problem_type="min", debug=True)

    if solution1 is not None:
        constraint_value = np.dot(A1_row, solution1)
        constraint_error = constraint_value - b1_val

        objective_value_check = np.dot(c1, solution1)
        objective_error = objective_value_check - objective_value1

        print("\n--- Example 1 Results ---")
        print(f"Optimal solution (x1, x2, x3): {tuple(solution1)}")
        print(f"Minimum objective value: {objective_value1:.10f}")
        print(f"Constraint check: {constraint_value:.20f} (Expected: {b1_val:.20f})")
        print(f"Constraint absolute error: {constraint_error:.20e}")
        print(f"Objective check: {objective_value_check:.20f} (Expected: {objective_value1:.20f})")
        print(f"Objective absolute error: {objective_error:.20e}")
    else:
        print("\n--- Example 1: No solution found ---")

    print("\n" + "="*50 + "\n")

    print("--- Example 2: Trying another problem ---")
    c2 = [2.0, 1.0]
    A2_row = [3.0, 2.0]
    b2_val = 6.0
    solution2, objective_value2 = solve_single_constraint_lp(c2, A2_row, b2_val, problem_type="min", debug=True)

    if solution2 is not None:
        constraint_value = np.dot(A2_row, solution2)
        constraint_error = constraint_value - b2_val

        objective_value_check = np.dot(c2, solution2)
        objective_error = objective_value_check - objective_value2

        print("\n--- Example 2 Results ---")
        print(f"Optimal solution (x1, x2): {tuple(solution2)}")
        print(f"Minimum objective value: {objective_value2:.10f}")
        print(f"Constraint check: {constraint_value:.20f} (Expected: {b2_val:.20f})")
        print(f"Constraint absolute error: {constraint_error:.20e}")
        print(f"Objective check: {objective_value_check:.20f} (Expected: {objective_value2:.20f})")
        print(f"Objective absolute error: {objective_error:.20e}")
    else:
        print("\n--- Example 2: No solution found ---")

    print("\n" + "="*50 + "\n")

    print("--- Example 3: Try a max problem instead ---")
    c3 = [5.0, 3.0]
    A3_row = [1.0, 2.0]
    b3_val = 10.0
    solution3, objective_value3 = solve_single_constraint_lp(c3, A3_row, b3_val, problem_type="max", debug=True)

    if solution3 is not None:
        constraint_value = np.dot(A3_row, solution3)
        constraint_error = constraint_value - b3_val

        objective_value_check = np.dot(c3, solution3)
        objective_error = objective_value_check - objective_value3

        print("\n--- Example 3 Results ---")
        print(f"Optimal solution (x1, x2): {tuple(solution3)}")
        print(f"Maximum objective value: {objective_value3:.10f}")
        print(f"Constraint check: {constraint_value:.20f} (Expected: {b3_val:.20f})")
        print(f"Constraint absolute error: {constraint_error:.20e}")
        print(f"Objective check: {objective_value_check:.20f} (Expected: {objective_value3:.20f})")
        print(f"Objective absolute error: {objective_error:.20e}")
    else:
        print("\n--- Example 3: No solution found ---")

--- Example 1: Original Problem ---
INFO: Choosing x1 as initial basic variable.

--- Iteration 0 ---
Tableau (Basic Var: x1):
          x1 |     x2 |     x3 |    RHS
 z' | 0.000000 | 1.000000 | 2.000000 | -1.000000
 x1 | 1.000000 | 1.000000 | 1.000000 | 1.000000

Optimal solution found.

--- Example 1 Results ---
Optimal solution (x1, x2, x3): (np.float64(1.0), np.float64(0.0), np.float64(0.0))
Minimum objective value: 1.0000000000
Constraint check: 1.00000000000000000000 (Expected: 1.00000000000000000000)
Constraint absolute error: 0.00000000000000000000e+00
Objective check: 1.00000000000000000000 (Expected: 1.00000000000000000000)
Objective absolute error: 0.00000000000000000000e+00


--- Example 2: Trying another problem ---
INFO: Choosing x1 as initial basic variable.

--- Iteration 0 ---
Tableau (Basic Var: x1):
          x1 |     x2 |    RHS
 z' | 0.000000 | -0.333333 | -4.000000
 x1 | 1.000000 | 0.666667 | 2.000000
Pivot column (entering var): x2 (index 1) with coeff -0.333333


In [6]:
print("\n" + "="*50 + "\n")

print("--- Example 4: Stress Test Problem ---")
# minimize 0.000001x1 + 1000000x2
# subject to 0.000003x1 + 2x2 = 6
c4 = [0.000001, 1000000.0]
A4_row = [0.000003, 2.0]
b4_val = 6.0
solution4, objective_value4 = solve_single_constraint_lp(c4, A4_row, b4_val, problem_type="min", debug = True)

if solution4 is not None:
    constraint_value = np.dot(A4_row, solution4)
    constraint_error = constraint_value - b4_val

    objective_value_check = np.dot(c4, solution4)
    objective_error = objective_value_check - objective_value4

    print("\n--- Example 4 Results ---")
    print(f"Optimal solution (x1, x2): {tuple(solution4)}")
    print(f"Minimum objective value: {objective_value4:.10f}")
    print(f"Constraint check: {constraint_value:.10f} (Expected: {b4_val:.10f})")
    print(f"Constraint absolute error: {constraint_error:.2e}")
    print(f"Objective check: {objective_value_check:.10f} (Expected: {objective_value4:.10f})")
    print(f"Objective absolute error: {objective_error:.2e}")
else:
    print("\n--- Example 4: No solution found ---")




--- Example 4: Stress Test Problem ---
INFO: Choosing x1 as initial basic variable.

--- Iteration 0 ---
Tableau (Basic Var: x1):
          x1 |     x2 |    RHS
 z' | 0.000000 | 999999.333333 | -2.000000
 x1 | 1.000000 | 666666.666667 | 2000000.000000

Optimal solution found.

--- Example 4 Results ---
Optimal solution (x1, x2): (np.float64(2000000.0), np.float64(0.0))
Minimum objective value: 2.0000000000
Constraint check: 6.0000000000 (Expected: 6.0000000000)
Constraint absolute error: 0.00e+00
Objective check: 2.0000000000 (Expected: 2.0000000000)
Objective absolute error: 0.00e+00


In [7]:
print("\n" + "="*50 + "\n")

print("--- Example 5: Ill-conditioned Problem ---")
# minimize 0.000001x1 + 1000000x2
# subject to 0.000003x1 + 2x2 = 6
c5 = [0.000001, 1000000.0]
A5_row = [0.000003, 2.0]
b5_val = 6.0
solution5, objective_value5 = solve_single_constraint_lp(c5, A5_row, b5_val, problem_type="min", debug=True)

if solution5 is not None:
    constraint_value = np.dot(A5_row, solution5)
    constraint_error = constraint_value - b5_val

    objective_value_check = np.dot(c5, solution5)
    objective_error = objective_value_check - objective_value5

    print("\n--- Example 5 Results ---")
    print(f"Optimal solution (x1, x2): {tuple(solution5)}")
    print(f"Minimum objective value: {objective_value5:.20f}")
    print(f"Constraint check: {constraint_value:.20f} (Expected: {b5_val:.20f})")
    print(f"Constraint absolute error: {constraint_error:.20e}")
    print(f"Objective check: {objective_value_check:.20f} (Expected: {objective_value5:.20f})")
    print(f"Objective absolute error: {objective_error:.20e}")
else:
    print("\n--- Example 5: No solution found ---")

print("\n" + "="*50 + "\n")

print("--- Example 6: Catastrophic Cancellation Problem ---")
# minimize (1+1e-8)x1 + (1-1e-8)x2
# subject to (1+1e-8)x1 + (1-1e-8)x2 = 2
c6 = [1.0 + 1e-8, 1.0 - 1e-8]
A6_row = [1.0 + 1e-8, 1.0 - 1e-8]
b6_val = 2.0
solution6, objective_value6 = solve_single_constraint_lp(c6, A6_row, b6_val, problem_type="min", debug=True)

if solution6 is not None:
    constraint_value = np.dot(A6_row, solution6)
    constraint_error = constraint_value - b6_val

    objective_value_check = np.dot(c6, solution6)
    objective_error = objective_value_check - objective_value6

    print("\n--- Example 6 Results ---")
    print(f"Optimal solution (x1, x2): {tuple(solution6)}")
    print(f"Minimum objective value: {objective_value6:.20f}")
    print(f"Constraint check: {constraint_value:.20f} (Expected: {b6_val:.20f})")
    print(f"Constraint absolute error: {constraint_error:.20e}")
    print(f"Objective check: {objective_value_check:.20f} (Expected: {objective_value6:.20f})")
    print(f"Objective absolute error: {objective_error:.20e}")
else:
    print("\n--- Example 6: No solution found ---")




--- Example 5: Ill-conditioned Problem ---
INFO: Choosing x1 as initial basic variable.

--- Iteration 0 ---
Tableau (Basic Var: x1):
          x1 |     x2 |    RHS
 z' | 0.000000 | 999999.333333 | -2.000000
 x1 | 1.000000 | 666666.666667 | 2000000.000000

Optimal solution found.

--- Example 5 Results ---
Optimal solution (x1, x2): (np.float64(2000000.0), np.float64(0.0))
Minimum objective value: 2.00000000000000000000
Constraint check: 6.00000000000000000000 (Expected: 6.00000000000000000000)
Constraint absolute error: 0.00000000000000000000e+00
Objective check: 2.00000000000000000000 (Expected: 2.00000000000000000000)
Objective absolute error: 0.00000000000000000000e+00


--- Example 6: Catastrophic Cancellation Problem ---
INFO: Choosing x1 as initial basic variable.

--- Iteration 0 ---
Tableau (Basic Var: x1):
          x1 |     x2 |    RHS
 z' | 0.000000 | 0.000000 | -2.000000
 x1 | 1.000000 | 1.000000 | 2.000000

Optimal solution found.

--- Example 6 Results ---
Optimal solu